# The Stencil operator: `⌺`

> Every time I see some piece of medical research saying that caffeine is good for you, I high-five myself. Because I'm going to live forever.
\--_Linus Torvalds_

Are you familiar with Conway's [Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)? If not, it's a cellular automata with a deceptively simple set of rules:

1. Any live cell with two or three live neighbours survives.
2. Any dead cell with three live neighbours becomes a live cell.
3. All other live cells die in the next generation.

yet G-of-L gives rise to extraordinary complex patterns. It's fun to implement -- try it in your favourite language! Some good ones can be found on [Rosettacode](https://rosettacode.org/wiki/Conway's_Game_of_Life).

In [1]:
⎕IO ← 0
{}⎕SE.UCMD'box on -s=min -t=tree -f=on'
{}⎕SE.UCMD'rows on'

done

Game of Life is a bit of an APL party trick. Here's the current champion, all 17 symbols of it:

In [2]:
gol←{≢⍸⍵}⌺3 3∊¨3+0,¨⊢ ⍝ Generate the next generation

To run it, we first need an array with some live cells in it. Here's an 8×8 petri dish containing a "glider":

In [3]:
glider ← 3 3⍴0 0 1 1 0 1 0 1 1
dish ← 8 8⍴0
dish[(⊂4 3)+⍸glider]←1
dish

0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0
0 0 0 0 0 1 0 0
0 0 0 1 0 1 0 0
0 0 0 0 1 1 0 0
0 0 0 0 0 0 0 0

The glider pattern is called that because it glides across the dish. Here's a full cycle:

In [4]:
1 4⍴(gol⍣1⊢dish)(gol⍣2⊢dish)(gol⍣3⊢dish)(gol⍣4⊢dish)

┌───────────────┬───────────────┬───────────────┬───────────────┐
│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│
│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│
│0 0 0 0 1 0 0 0│0 0 0 0 0 1 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│
│0 0 0 0 0 1 1 0│0 0 0 0 0 0 1 0│0 0 0 0 1 0 1 0│0 0 0 0 0 0 1 0│
│0 0 0 0 1 1 0 0│0 0 0 0 1 1 1 0│0 0 0 0 0 1 1 0│0 0 0 0 1 0 1 0│
│0 0 0 0 0 0 0 0│0 0 0 0 0 0 0 0│0 0 0 0 0 1 0 0│0 0 0 0 0 1 1 0│
└───────────────┴───────────────┴───────────────┴───────────────┘

Anyway -- this APL G-of-L implementation is primarily a demonstration of the power and versatility of the [Stencil](http://help.dyalog.com/18.0/index.htm#Language/Primitive%20Operators/Stencil.htm) operator, `⌺`. In simple terms, the dyadic stencil operator lets you specify a sliding window of a rank of your choice to the right and a function operating over this sliding window to the left. It makes it trivial to implement image processing filters, or convolution. In essence, for the 2-D case, it's four nested for-loops, the outermost two moving the window, and the innermost two visiting every data point within the window.

This is obviously a good fit for Game of Life: the rules are defined within a 3×3 neighbourhood, so all we need to do is count the 1s in the windows that stencil delivers:

In [5]:
{≢⍸⍵} glider

5

Let's look at stencil in a bit more detail. We can illustrate the sliding window using enclose as the left operand and a size-2 window over a vector:

In [6]:
{⊂⍵}⌺2⊢⍳10

┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
│0 1│1 2│2 3│3 4│4 5│5 6│6 7│7 8│8 9│
└───┴───┴───┴───┴───┴───┴───┴───┴───┘

But now look what happens if we increase the window size to 3, which does not divide cleanly the operand array:

In [7]:
{⊂⍵}⌺3⊢⍳10

┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐
│0 0 1│0 1 2│1 2 3│2 3 4│3 4 5│4 5 6│5 6 7│6 7 8│7 8 9│8 9 0│
└─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘

The first and last windows now "overhang" the edge, filling the overhang with zeros. The way to think of it is that given an odd-size window, each data point in the argument array must line up with the _centre_ of the window. With even-sized windows, we have no choice, as we can't have fractional indexing.

Stencil has a few more tricks up its sleeve. The right operand can be an array, where the first row defines the shape of the window, and the second row defines the _step size_. Again, using the vector above, we can say that we want a size 3 window, but instead of moving it 1 step (the default) we want to shift it 3 steps each time:

In [8]:
{⊂⍵}⌺(2 1⍴3 3)⊢⍳10

┌─────┬─────┬─────┬─────┐
│0 0 1│2 3 4│5 6 7│8 9 0│
└─────┴─────┴─────┴─────┘

We still have an odd-size window, and the first is centered over the first item as before. The step size, 3, dictates how far the center of the window moves, so the next window is centered over item 3 etc.

With an even size we can create non-overlapping windows that don't "overhang":

In [9]:
{⊂⍵}⌺(2 1⍴2 2)⊢⍳10

┌───┬───┬───┬───┬───┐
│0 1│2 3│4 5│6 7│8 9│
└───┴───┴───┴───┴───┘